In [ ]:
# Description : This program uses an artificial recurrent neural network called
# Long short Term Memory LSTM
# to predict the closing stock price of a corporation using the past 60 day stock price.
#import the library
import math
import pandas_datareader as web
import numpy as np
import pandas as pd 
import io
from datetime import date, timedelta
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight')
 
#List of ticker for Nifty 50 Companies.
ticker_list = ['RELIANCE.NS','HDFCBANK.NS','INFY.NS','HDFC.NS','TCS.NS','ICICIBANK.NS','KOTAKBANK.NS','HINDUNILVR.NS','ITC.BO','BHARTIARTL.NS','LT.NS','AXISBANK.NS','BAJFINANCE.NS','MARUTI.NS','ASIANPAINT.NS','HCLTECH.NS','SBIN.NS','NESTLEIND.NS','M&M.NS','SUNPHARMA.NS','DRREDDY.NS']
 
#Function will return the dataset with future predictions of stock prices of next 1 - year.
#1825 stands for 5 year time period.
def predict_stock_prices(company_ticker,prediction_based_on_past_days = 180):
  """
    Explaination of function.
  """
 
  #Getting start and end date to give input to yahoo api.
  today = date.today()
  current_date = today.strftime("%m%d%Y")  
  month_old_date = (date.today()-timedelta(days=prediction_based_on_past_days)).strftime("%m%d%Y") 
  from_day = month_old_date[2:4]
  from_month = month_old_date[0:2]
  from_year = month_old_date[4:]
  to_day = current_date[2:4]
  to_month = current_date[0:2]  
  to_year = current_date[4:]
  start_date = from_year + "-" + from_month + "-" + from_day
  end_date = to_year + "-" + to_month + "-" + to_day
  
  
  #Retriving data from yahoo api.
  df = web.DataReader(company_ticker,data_source = 'yahoo',start = start_date,end = end_date)
  # df = df.set_index('Date')
  # df = df.iloc[::-1]
 
 
  # Create a new dataframe with only the close column
  data = df.filter(['Close'])
  # Convert the dataframe to a numpy array
  dataset = data.values
  print(dataset)
  #Get the number of rows to train the model on
  training_data_len = math.ceil(len(dataset) * 1)
 
  #Scale the data
  #its just a good practice to normalize or to scale the data before giving it to neural network.
  scaler = MinMaxScaler(feature_range = (0,1))
  #feature range tells us in which range the data lies.
  scaled_data = scaler.fit_transform(dataset)
 
  #Create the training data set
  #Create the scaled training data set
  train_data = scaled_data[0:training_data_len,:]
  #Split the data into x_train and y_train data sets
  x_train = []
  y_train = []
  for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
 
  #Convert the x_train and y_train to numpy arrays
  x_train, y_train = np.array(x_train) , np.array(y_train)
  #Reshape the data
  # An Lstm model expects the input  as 3-d data ,no_of_rows(no of samples) time-stamp and number of feature
  x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
  
 
  # Build the LSTM
  model = Sequential()
  model.add(LSTM(50,return_sequences = True, input_shape = (x_train.shape[1],1)))
  model.add(LSTM(50,return_sequences=False))
  model.add(Dense(25))
  model.add(Dense(1))
  
  #Compile the model 
  #optimizer is used to improve upon loss function
  #loss function used to calculate how welll the model did on training.
  model.compile(optimizer = 'adam', loss = 'mean_squared_error')
  
  #Train the model
  model.fit(x_train,y_train,batch_size = 1,epochs = 1)
  #get the quote
  apple_quote = web.DataReader(company_ticker,data_source = 'yahoo',start = start_date,end = end_date)
  #create a new dataframe
  new_df = apple_quote.filter(['Close'])
  #get the last 60 day closing price values and convert the dataframe to an array
  last_60_days = new_df[-60:].values
  #scale the data to be values b/w 0 and 1
  last_60_days_scaled = scaler.transform(last_60_days)
  #create a empty list
  X_test_ = []
  #Append teh last_60_days_scaled
  X_test_.append(last_60_days_scaled)
  X_test_ = np.array(X_test_)
  X_test__ = np.copy(X_test_)
  counter = 365
  while(counter > 0):
    X_test_ = X_test__[0][-60:]
    X_final = []
    X_final.append(X_test_)
    X_final = np.array(X_final)
    X_final = np.reshape(X_final, (X_final.shape[0],X_final.shape[1],1))
    # Get the predicted scaled price
    pred_price = model.predict(X_final)
    x_t = np.append(X_test__[0],pred_price).reshape((X_test__[0].shape[0] + pred_price.shape[0],1))
    l = []
    l.append(x_t)
    l = np.array(l)
    X_test__ = np.reshape(l,(l.shape[0],l.shape[1],1))
    counter = counter - 1
  arr = X_test__
  arr = arr.reshape(arr.shape[1],1)
  lis = []
  for i in range(0,arr.shape[0]):
    lis.append(scaler.inverse_transform(np.array(arr.item(i,0)).reshape(-1,1)).item(0,0))
  return lis

dict = {
    
}

for company in ticker_list:
  lis = predict_stock_prices(company)
  dict[company] = lis

df = pd.DataFrame(dict)
print(df.head(50))    
print(df.shape)
    

[[1561.98999023]
 [1465.35998535]
 [1496.44995117]
 [1435.94995117]
 [1459.40002441]
 [1440.75      ]
 [1408.90002441]
 [1433.69995117]
 [1441.25      ]
 [1431.55004883]
 [1424.05004883]
 [1445.55004883]
 [1472.25      ]
 [1464.40002441]
 [1520.34997559]
 [1535.69995117]
 [1541.65002441]
 [1579.80004883]
 [1581.69995117]
 [1569.5       ]
 [1537.15002441]
 [1572.15002441]
 [1537.69995117]
 [1588.80004883]
 [1614.55004883]
 [1617.69995117]
 [1615.34997559]
 [1655.90002441]
 [1759.40002441]
 [1746.15002441]
 [1720.90002441]
 [1727.84997559]
 [1717.90002441]
 [1741.65002441]
 [1723.15002441]
 [1704.09997559]
 [1737.59997559]
 [1760.34997559]
 [1787.90002441]
 [1851.80004883]
 [1823.44995117]
 [1798.        ]
 [1824.25      ]
 [1878.05004883]
 [1935.        ]
 [1917.        ]
 [1844.        ]
 [1843.40002441]
 [1911.69995117]
 [1919.90002441]
 [1971.55004883]
 [2004.        ]
 [2057.80004883]
 [2146.14990234]
 [2156.19995117]
 [2177.69995117]
 [2096.64990234]
 [2108.85009766]
 [2067.1000976

In [ ]:
dic=[]
val=[]
fin=[]
df1=[]
df3=[]
df6=[]
df9=[]
df12=[]
print(ticker_list)
print(df)
for company in ticker_list:
  dic.append(company)
  print(df[company][180])
  diff1=(df[company][30]-df[company][0])/df[company][0]
  #print(diff1)
  diff1*=100
  df1.append(diff1)
  val.append(diff1)
  diff3=(df[company][90]-df[company][0])/df[company][0]
  diff3*=100
  df3.append(diff3)
  val.append(diff3)

  diff6=(df[company][180]-df[company][0])/df[company][0]
  diff6*=100
  val.append(diff6)
  df6.append(diff6)
  diff9=(df[company][270]-df[company][0])/df[company][0]
  diff9*=100
  val.append(diff9)
  df9.append(diff9)
  print(df[company][270])

  diff12=(df[company][365]-df[company][0])/df[company][0]
  diff12*=100
  val.append(diff12)
  df12.append(diff12)
  fin.append(val)
  val=[]
fin

['RELIANCE.NS', 'HDFCBANK.NS', 'INFY.NS', 'HDFC.NS', 'TCS.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'HINDUNILVR.NS', 'ITC.BO', 'BHARTIARTL.NS']
     RELIANCE.NS  HDFCBANK.NS  ...      ITC.BO  BHARTIARTL.NS
0    2113.800049  1034.449951  ...  196.500000     528.599976
1    2091.350098  1032.750000  ...  198.649994     521.400024
2    2118.550049  1056.500000  ...  198.600006     526.299988
3    2131.550049  1066.599976  ...  198.449997     535.849976
4    2097.050049  1059.000000  ...  195.850006     525.549988
..           ...          ...  ...         ...            ...
420  2214.593014  1374.321239  ...  165.862325     411.657453
421  2214.592959  1374.321864  ...  165.862325     411.657453
422  2214.592959  1374.322376  ...  165.862325     411.657453
423  2214.592959  1374.323002  ...  165.862325     411.657453
424  2214.592959  1374.323457  ...  165.862325     411.657453

[425 rows x 10 columns]
2214.363954398643
2214.5914313114845
1366.2442150115967
1373.4176886081696
1073.442283190193

[[4.158383021728145,
  3.19158486895154,
  4.7574937670320185,
  4.768255282198402,
  4.7683301583835815],
 [0.9232006649820266,
  26.568280242015664,
  32.074462709756915,
  32.76792048298668,
  32.84863168108054],
 [6.066488894942339,
  13.081069814545273,
  12.567356404410507,
  12.550095391171034,
  12.549501382295139],
 [-7.797061727913926,
  3.8620155066707795,
  2.3138774708289733,
  2.3058506231050497,
  2.305808287410303],
 [8.034705721328516,
  25.826322449607098,
  31.038147697933006,
  31.499071835376853,
  31.534486146698093],
 [-3.527946932852476,
  9.90009095941308,
  8.12037343326524,
  8.081611989806001,
  8.08074256012045],
 [-4.160763652558252,
  6.9192489802530694,
  4.379527722722472,
  4.3732744165762805,
  4.373263395106976],
 [-4.363119636436084,
  -1.8059577391275246,
  -1.6040422741224662,
  -1.6030018350925306,
  -1.6029953541594262],
 [-13.078878854067282,
  -15.258586145420711,
  -15.587589827711918,
  -15.59164106745132,
  -15.591691499360206],
 [-16.82746

In [ ]:
dfx = pd.DataFrame(list(zip(dic,df1,df3,df6,df9,df12)), 
               columns =['Name', '1_month','3_months','6_months','9_months','12_months']) 
dfx

,Name,1_month,3_months,6_months,9_months,12_months
0,RELIANCE.NS,4.158383,3.191585,4.757494,4.768255,4.768330
1,HDFCBANK.NS,0.923201,26.568280,32.074463,32.767920,32.848632
2,INFY.NS,6.066489,13.081070,12.567356,12.550095,12.549501
3,HDFC.NS,-7.797062,3.862016,2.313877,2.305851,2.305808
4,TCS.NS,8.034706,25.826322,31.038148,31.499072,31.534486
5,ICICIBANK.NS,-3.527947,9.900091,8.120373,8.081612,8.080743
6,KOTAKBANK.NS,-4.160764,6.919249,4.379528,4.373274,4.373263
7,HINDUNILVR.NS,-4.363120,-1.805958,-1.604042,-1.603002,-1.602995
8,ITC.BO,-13.078879,-15.258586,-15.587590,-15.591641,-15.591691
9,BHARTIARTL.NS,-16.827466,-21.020000,-22.116639,-22.123031,-22.123066


In [ ]:
dfx.to_csv('file_name1.csv', index=False)
